In [90]:
import pandas
pandas.__version__

u'0.21.0'

## Creating 3 Dict
- First 3 are in form inps[id->(tx_id,output_id)]
- The remaining dictionaries are inps_by_tx being tx_id->[inp_id]* where inps[inp_id][0] = tx_id

In [91]:
from collections import defaultdict
inps = {}
outs = {}
txs = {}

def readIntoDict(d,f_name,key_index):
    with open(f_name,'r') as f:
        for l in f:
            arr=l.split(',')
            k = int(arr[key_index])
            del arr[key_index]
            d[k] = tuple(map(int,arr))
readIntoDict(inps,'inputs.csv',0)
readIntoDict(outs,'outputs.csv',0)
readIntoDict(txs,'transactions.csv',0)


In [92]:
inps_by_tx = defaultdict(list)
inps_by_out = defaultdict(list)
outs_by_tx = defaultdict(list)
txs_by_blk = defaultdict(list)

def groupByFirstColumn(dict_new,dict_id,i=0):
    for k,v in dict_id.iteritems():
          dict_new[v[i]].append(k)
groupByFirstColumn(inps_by_tx,inps)
groupByFirstColumn(inps_by_out,inps,i=1)
groupByFirstColumn(outs_by_tx,outs)
groupByFirstColumn(txs_by_blk,txs)


## Block Validty Checker


In [93]:
class Checker(object):
    def __init__(self):
        self.txo = set() #outputs that is spent
    
    def checkInputs(s,t_id,block_id):
        tx_inputs = [inps[i][-1] for i in inps_by_tx[t_id]]
        tx_outputs = [o for o in outs_by_tx[t_id]]
        if len(set(tx_inputs) & set(tx_outputs) ) != 0:
            print 'ERROR:Output transaction cannot be input'
            print t_id,tx_inputs,tx_outputs,block_id
            return False
        if len(tx_inputs) != len(set(tx_inputs)): 
            print 'ERROR:There are two inputs pointing the same previous output'
            print t_id,tx_inputs,block_id
            return False
        for o in tx_inputs:
            if o not in outs:
                print 'ERROR:Output script is not correct/doesn\'t exists' 
                print t_id,tx_inputs,block_id
                return False
        for o in tx_outputs:
            if outs[o][-1]<0:
                print 'ERROR:Output value cant be less <0.'
                print t_id,tx_outputs,block_id
                return False
        return True
    
    def checkBlockSum(s,tx_ids,block_id):
        output_sum = input_sum = 0
        curr_txo = set()
        for t_id in tx_ids:
            tx = txs[t_id]
            if not s.checkInputs(t_id,block_id):
                return False
            tx_inputs = set(inps_by_tx[t_id])
            tx_outputs = set(outs_by_tx[t_id])
            for i_id in tx_inputs:
                prev_out = inps[i_id][-1]
                if prev_out in s.txo:
                    print 'ERROR:output %d already used in another block. Called from i_id: %d,t_id: %d,block_id: %d' % (prev_out,i_id,t_id,block_id)
                    return False
                elif prev_out in curr_txo: #
                    print 'ERROR:output %d already used within same block. Called from i_id: %d,t_id: %d,block_id: %d' % (prev_out,i_id,t_id,block_id)
                    return False
                else:
                    input_sum += outs[prev_out][-1] #[1] for output_id, [-1] for value
                    curr_txo.add(prev_out)
                   
            for o_id in tx_outputs:
                    output_sum += outs[o_id][-1] #[-1] for value
        if output_sum - input_sum > 5000000000:
            print 'ERROR:Sum is not correct'
            print output_sum - input_sum 
            return False
        else:
            s.txo.update(set([inps[i][-1] for i in tx_inputs]))
            return True
    def checkBlockBasic(s,block_id):
        tx_ids = txs_by_blk[block_id]

        if len(set(tx_ids)) != len(tx_ids):
            print 'ERROR:Error multiple txs with same id!'
            print tx_ids
            return False
        coinbase_txs = [i for i in tx_ids if txs[i][-1]==1]
        if len(coinbase_txs) != 1:
            print 'ERROR:Multiple coinbase transactions!'
            print coinbase_txs
            return False
        coinbase_output_sum = sum([outs[o][-1] for o in (outs_by_tx[coinbase_txs[0]])])
        if coinbase_output_sum < 5000000000:
            print 'WARNING coinbase sum is less than the reward, wth might be wrong', coinbase_txs[0]
        if inps_by_tx[coinbase_txs[0]]:
            print 'ERROR:Coinbase transactions can\'t have input!'
            print 'tx: %d, b_id: %d ' % (coinbase_txs[0],block_id)
            return False
        if not s.checkBlockSum(tx_ids,block_id):
            return False
        return True
                

In [94]:
from collections import Counter
cc = Checker()
invalidBlocks = set()
for k in sorted(txs_by_blk.keys()):
    if not cc.checkBlockBasic(k):
        print k,txs_by_blk[k]
        print '---------'
        invalidBlocks.add(k)
#print cc.txo

ERROR:Sum is not correct
5000000010
11181 [11281, 11282]
---------
ERROR:output 7998 already used in another block. Called from i_id: 777,t_id: 12152,block_id: 12042
12042 [12151, 12152]
---------
ERROR:Output script is not correct/doesn't exists
15698 [265834] 15567
15567 [15697, 15698]
---------
ERROR:There are two inputs pointing the same previous output
30446 [21928, 21928] 30223
30223 [30445, 30446]
---------
ERROR:Multiple coinbase transactions!
[56851, 56853]
52534 [56851, 56852, 56853]
---------
ERROR:output 65403 already used within same block. Called from i_id: 20183,t_id: 61845,block_id: 56565
56565 [61841, 61842, 61843, 61844, 61845]
---------
ERROR:Sum is not correct
5000000010
72902 [100928, 100929]
---------
ERROR:Output value cant be less <0.
105281 [123672] 75047
75047 [105279, 105280, 105281]
---------
ERROR:Output transaction cannot be input
137237 [156879, 166481] [166481, 166482] 88755
88755 [137236, 137237]
---------
ERROR:output 249860 already used in another blo

## Graph Coloring
- What we want to do give colours to the each transaction, such that each related transaction should have the same -colour. 
- Then each colour groups would identify a person, we can 

In [95]:
class GraphSearch(object):
    def __init__(s,invalid_blocks=set()):
        s.invalid_blocks = invalid_blocks
        s.reset_search()
    def reset_search(s):
        s.seen_transactions = set()
        for inv_b in s.invalid_blocks:
            s.seen_transactions.update(txs_by_blk[inv_b])
        s.utxo = set()
    def dfs(s):
        def dfs_helper(tx_id):
            if tx_id not in s.seen_transactions:
                #print tx_id
                is_coinbase = txs[tx_id][1]
                if is_coinbase:
                    s.utxo.update(outs_by_tx[tx_id])
                else:
                    tx_inputs_outs = [inps[i][-1] for i in inps_by_tx[tx_id]]
                    for tx_inp_out in tx_inputs_outs:
                        dfs_helper(outs[tx_inp_out][0]) #tx of the output input points to. 
                        if tx_inp_out not in s.utxo:
                            print 'ERROR input cannot be found, tx_id,tx_inp:%d,%d' %(tx_id,tx_inp_out)
                    s.utxo.update(outs_by_tx[tx_id])
                    s.utxo.difference_update(tx_inputs_outs)
                s.seen_transactions.add(tx_id)
                            
        for tx_id in txs.keys():
            dfs_helper(tx_id)


In [96]:
gs = GraphSearch(invalidBlocks)
gs.dfs()

ERROR input cannot be found, tx_id,tx_inp:114916,60228
ERROR input cannot be found, tx_id,tx_inp:204751,249860
ERROR input cannot be found, tx_id,tx_inp:207370,249869
ERROR input cannot be found, tx_id,tx_inp:208063,249859
ERROR input cannot be found, tx_id,tx_inp:208084,249863
ERROR input cannot be found, tx_id,tx_inp:208477,249852
ERROR input cannot be found, tx_id,tx_inp:208644,249853
ERROR input cannot be found, tx_id,tx_inp:209913,249855
ERROR input cannot be found, tx_id,tx_inp:211792,249857
ERROR input cannot be found, tx_id,tx_inp:213613,249856


In [97]:
print len(sorted([(outs[a][-1],a) for a in gs.utxo],reverse=True))

71889


In [98]:
class EntityGroupingSearch(object):
    def __init__(s,invalid_blocks=set()):
        s.invalid_blocks = invalid_blocks
        s.reset_search()
    def reset_search(s):
        s.txlabels = {}
        #Prelabeling invalid tx's
        for inv_b in s.invalid_blocks:
            s.txlabels.update({tx:-1 for tx in txs_by_blk[inv_b]})
        
    def dfs(s):
        def dfs_helper(tx_id,label_id):
            if tx_id not in s.txlabels:
                if len(outs_by_tx[tx_id]) == 1: #one owner!
                    s.txlabels[tx_id] = label_id
                    dfs_helper(outs[outs_by_tx[tx_id][0]][0],label_id)
                    for i in inps_by_tx[tx_id]:
                        dfs_helper(inps[i][0],label_id)
                else:
                    s.txlabels[tx_id] = 0                    
        i = 0
        for tx_id in txs.keys():
            if tx_id not in s.txlabels:
                i+=1
                dfs_helper(tx_id,i)


In [99]:
egs = EntityGroupingSearch(invalidBlocks)
egs.dfs()

## Recursively matching public keys and outputs
Whenever we process the output we merge it, if we seen it before

In [100]:
from collections import defaultdict
entities = defaultdict(list)
pk_to_entity = {}
for k,v in egs.txlabels.iteritems():
    if v > 0:
        prev_outs = [inps[i][-1] for i in inps_by_tx[k]]
        for o in outs_by_tx[k] + prev_outs:
            pk = outs[o][1]
            if pk in pk_to_entity:
                new_ent = pk_to_entity[pk]
                if new_ent != v:
                    if v in entities: # merge the current entity to the other
                        for old_pk in entities[v]:
                            pk_to_entity[old_pk] = new_ent
                        entities[new_ent].extend(entities[v])
                        del entities[v]
                    v = new_ent
            else:
                entities[v].append(pk)
                pk_to_entity[pk] = v
                


## Clasifying entities

In [101]:
from collections import defaultdict
entity_utxos = defaultdict(list)
for o in gs.utxo:
    if outs[o][1] in pk_to_entity:
        entity_utxos[pk_to_entity[outs[o][1]]].append(o)
print len(entity_utxos)


53738


## Calculating balances

In [102]:
person_total_balances = {}
for e,os in entity_utxos.iteritems():    
    person_total_balances[e] = sum([outs[o][-1] for o in os])

##  Total Value

In [76]:
print sorted([(v,k) for k,v in person_total_balances.items()],reverse=True)[0]

(98346739827397, 215565)

In [80]:
print 'Minimum publickey is %d ' % min(entities[215565])

Minimum publickey is 172 


In [103]:
my_pks = set(entities[215565])

In [104]:
max(((o_res[-1],o_res[0]) for o_id,o_res in outs.items() if o_res[1] in my_pks))
    

(5500000000000, 139532)

In [88]:
print len(outs.items())

264310
